In [1]:
import h5py
import rebound
import numpy as np
import pandas as pd
import dask.dataframe as dd
from celmech import Andoyer
from spock import FeatureClassifier

# This requires the forecaster package(Chen & Kipping arXiv:1603.08614) 
# For a Python 3 compatiable version: https://github.com/bmorris3/forecaster
# You need to add the forecaster folder to your path below
import sys
sys.path.append('../forecaster/')
import mr_forecast as mr

# van Eylen 2015 figure sets are different in the arxiv and apj versions. arxiv matches folder sent by Vincent. Note that the numbers are in different order than planets

* 03 = b
* 01 = c
* 02 = d

# omega measured CCW from line of sight at which planet transits. rows in ecc list and w list match up in order 

In [2]:
eb = np.loadtxt('data/Kepler23vanEylen15/eccentricity_listK00168.03.txt')
wb = np.loadtxt('data/Kepler23vanEylen15/omega_listK00168.03.txt')
ec = np.loadtxt('data/Kepler23vanEylen15/eccentricity_listK00168.01.txt')
wc = np.loadtxt('data/Kepler23vanEylen15/omega_listK00168.01.txt')
ed = np.loadtxt('data/Kepler23vanEylen15/eccentricity_listK00168.02.txt')
wd = np.loadtxt('data/Kepler23vanEylen15/omega_listK00168.02.txt')

# We have to sample masses from the radii to calculate e_COM

Exact values don't matter since those constraints are much wider than TTVs or stability

In [3]:
Mearth = 3e-6
yr = 365.25 # days
Rsun = 1/215 # AU
Mstar = 1.078
Rstar = 1.548*Rsun
Pb = 7.107
Pc = 10.742
Pd = 15.274
ad = (Pd/yr)**(2/3)*(Mstar)**(1/3)

# Radii from DR 25 (see exomasses/eccentricities.ipynb)
Rb = 1.72 # +0.11 - 0.07 Rearth
Rc = 3.06 # +0.19 - 0.12 Rearth
Rd = 2.17 # +0.13 - 0.09 Rearth

imax = Rstar/ad # max inclination for outermost planet to transit
print(imax*180/np.pi)

def vanEylensim(ID):
    np.random.seed(ID)
    sim = rebound.Simulation()
    sim.units = ['days', 'AU', 'Msun']
    sim.add(m=Mstar)
    sim.add(m=mr.Rpost2M([Rb], unit='Earth', classify='No')[0]*Mearth, P=Pb, M=np.random.random()*2*np.pi, e=np.abs(eb[ID]), pomega=wb[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    sim.add(m=mr.Rpost2M([Rc], unit='Earth', classify='No')[0]*Mearth, P=Pc, M=np.random.random()*2*np.pi, e=np.abs(ec[ID]), pomega=wc[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    sim.add(m=mr.Rpost2M([Rd], unit='Earth', classify='No')[0]*Mearth, P=Pd, M=np.random.random()*2*np.pi, e=np.abs(ed[ID]), pomega=wd[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    return sim

def vanEylenrow(row): # Assumes inner two near 3:2 and outer two near 7:5 like in Kepler23
    sim = vanEylensim(row['ID'])
    P1 = sim.particles[1].P
    
    avars12 = Andoyer.from_Simulation(sim, j=3, k=1, i1=1, i2=2)
    row['Mstar'] = 1.078
    row['Z12'] = avars12.Z
    row['Zcom12'] = avars12.Zcom
    avars23 = Andoyer.from_Simulation(sim, j=7, k=2, i1=2, i2=3)
    row['Z23'] = avars23.Z
    row['Zcom23'] = avars23.Zcom
    row['e1'] = sim.particles[1].e
    row['e2'] = sim.particles[2].e
    row['e3'] = sim.particles[3].e
    row['mu1'] = sim.particles[1].m/sim.particles[0].m
    row['mu2'] = sim.particles[2].m/sim.particles[0].m
    row['mu3'] = sim.particles[3].m/sim.particles[0].m
    row['mu1+mu2'] = row['mu1'] + row['mu2']
    row['mu2+mu3'] = row['mu2'] + row['mu3']
    row['mutot'] = row['mu1'] + row['mu2'] + row['mu3']
    row['h1'] = sim.particles[1].e*np.sin(sim.particles[1].pomega)
    row['h2'] = sim.particles[2].e*np.sin(sim.particles[2].pomega)
    row['h3'] = sim.particles[3].e*np.sin(sim.particles[3].pomega)
    row['k1'] = sim.particles[1].e*np.cos(sim.particles[1].pomega)
    row['k2'] = sim.particles[2].e*np.cos(sim.particles[2].pomega)
    row['k3'] = sim.particles[3].e*np.cos(sim.particles[3].pomega)
    hcom = (row['mu1']*row['h1'] + row['mu2']*row['h2'] + row['mu3']*row['h3'])/(row['mu1']+row['mu2']+row['mu3'])
    kcom = (row['mu1']*row['k1'] + row['mu2']*row['k2'] + row['mu3']*row['k3'])/(row['mu1']+row['mu2']+row['mu3'])
    row['ecom'] = np.sqrt(hcom**2 + kcom**2)
    
    return row

3.3394414552197382


In [4]:
dfvanEylen = pd.DataFrame()
dfvanEylen['ID'] = range(40000)

In [5]:
%%time
dfvanEylen = dfvanEylen.apply(vanEylenrow, axis=1)

CPU times: user 18min 19s, sys: 5.7 s, total: 18min 25s
Wall time: 18min 25s


In [6]:
dfvanEylen.to_csv('csvs/Kepler-23_transitdurations.csv')

# TTVs

Generate a TTV csv if it doesn't already exist:

In [9]:
%%time
try:
    df = pd.read_csv("csvs/Kepler-23_TTVs.csv", index_col=0)
except:
    post_file = "data/Hadden17/NBody_MCMC_Posteriors.hdf5"
    post_data = h5py.File(post_file,"r")
    data = post_data['Kepler-23/DefaultPriors/PosteriorSample']
    npl = data.shape[1]//5
    col_heads = []
    for i in range(npl):
        col_heads += ['mu{}'.format(i+1),'per{}'.format(i+1),'h{}'.format(i+1),'k{}'.format(i+1),'Tc{}'.format(i+1)]
    df = pd.DataFrame(data=data,columns=col_heads)
    print(df.shape)
    
    for i in range(npl):
        h = df['h{}'.format(i+1)]
        k = df['k{}'.format(i+1)]
        per = df['per{}'.format(i+1)]
        Tc = df['Tc{}'.format(i+1)]
        df['pomega{}'.format(i+1)] = np.arctan2(k,h)
        df['e{}'.format(i+1)] = np.sqrt(h**2 + k**2)
        df['mean_long{}'.format(i+1)] = np.mod(-2 * np.pi * Tc / per,2*np.pi)
    hcom = (df['mu1']*df['h1'] + df['mu2']*df['h2'] + df['mu3']*df['h3'])/(df['mu1']+df['mu2']+df['mu3'])
    kcom = (df['mu1']*df['k1'] + df['mu2']*df['k2'] + df['mu3']*df['k3'])/(df['mu1']+df['mu2']+df['mu3'])
    df['ecom'] = np.sqrt(hcom**2 + kcom**2)

(250000, 15)
CPU times: user 29.7 s, sys: 248 ms, total: 29.9 s
Wall time: 27.6 s


In [10]:
def addZ(row):
    sim = rebound.Simulation()
    sim.units = ('Msun','days','AU')
    sim.add(m=1)
    for i in range(npl):
        mu = row['mu{}'.format(i+1)]
        per = row['per{}'.format(i+1)]
        e = row['e{}'.format(i+1)]
        mean_long = row['mean_long{}'.format(i+1)]
        pomega = row['pomega{}'.format(i+1)]
        sim.add(m=mu,P=per,e=e,l=mean_long,pomega=pomega,hash=i)
    sim.move_to_com()
    
    row['Mstar'] = 1.078
    avars12 = Andoyer.from_Simulation(sim, j=3, k=1, i1=1, i2=2)
    row['Z12'] = avars12.Z
    row['Zcom12'] = avars12.Zcom
    avars23 = Andoyer.from_Simulation(sim, j=7, k=2, i1=2, i2=3)
    row['Z23'] = avars23.Z
    row['Zcom23'] = avars23.Zcom
    
    return row

In [11]:
%%time
df = df.apply(addZ, axis=1)

CPU times: user 1h 1min 30s, sys: 13.2 s, total: 1h 1min 43s
Wall time: 1h 1min 40s


In [12]:
df.tail()

,mu1,per1,h1,k1,Tc1,mu2,per2,h2,k2,Tc2,...,mean_long2,pomega3,e3,mean_long3,ecom,Mstar,Z12,Zcom12,Z23,Zcom23
249995,0.000003,7.105870,-0.038511,-0.196506,57.031001,0.000006,10.743525,0.032910,-0.196457,55.659821,...,5.147323,-1.075444,0.199238,4.333169,0.195363,1.078,0.057635,0.225596,0.052881,0.202704
249996,0.000003,7.106110,-0.069445,0.025974,56.975209,0.000006,10.743671,0.000208,-0.003399,55.525812,...,5.226137,-0.406624,0.062597,4.431526,0.002368,1.078,0.048025,0.029160,0.047266,0.025114
249997,0.000004,7.106614,-0.001841,-0.075112,57.111239,0.000004,10.743291,0.044821,-0.081911,55.689775,...,5.129095,-0.580675,0.077984,4.414196,0.075583,1.078,0.039443,0.087027,0.029059,0.090064
249998,0.000003,7.106508,-0.087985,0.084049,56.927508,0.000004,10.743239,-0.016223,0.039809,55.475669,...,5.254157,0.894684,0.026970,4.517402,0.044281,1.078,0.048541,0.084225,0.025173,0.031786
249999,0.000004,7.106154,-0.122730,-0.000055,56.853333,0.000009,10.743234,-0.046245,-0.023290,55.364239,...,5.319310,-2.470655,0.004345,4.557613,0.066148,1.078,0.045537,0.083135,0.029900,0.047405


In [13]:
df.to_csv("csvs/Kepler-23_TTVs.csv")